In [1]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

tf.set_random_seed(777)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
import requests
url = 'https://biendata.com/competition/airquality/bj/2018-03-01-0/2018-05-31-23/2k0d1d8'
respones= requests.get(url)
with open ("bj_airquality.csv",'w') as f:
    f.write(respones.text)

In [12]:
import pandas as pd
b_air = pd.read_csv("bj_airquality.csv")

In [13]:
b_air = b_air[b_air['time'] < '2018-05-18 00:00:00'] # 19일,20일 예측이면 2018-05-18 00:00:00 까지 데이터 받아서 진행

In [15]:
b_air.tail(3)

,id,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
37167,3002367,xizhimenbei_aq,2018-05-16 23:00:00,76.0,78.0,62.0,1.5,26.0,3.0
37168,3002368,nansanhuan_aq,2018-05-16 23:00:00,29.0,92.0,69.0,1.2,23.0,7.0
37169,3002369,dongsihuan_aq,2018-05-16 23:00:00,131.0,NaN,59.0,1.7,17.0,10.0


In [16]:
b_air.columns = ['id', 'test_id', 'time', 'PM2.5', 'PM10','NO2', 'CO', 'O3', 'SO2']

In [17]:
sample = pd.read_csv("sample_submission.csv")

def id(i):
    return i.split('#')[0]

test_station = pd.DataFrame(list(map(id, sample['test_id']))).drop_duplicates()
test_station.columns = ['id']
test_station = list(test_station['id'])

bj_test_station = test_station[:35]
ld_test_station = test_station[35:]

In [18]:
def function1(i, y):
    b_air[b_air["test_id"] == i][y].fillna(np.mean(b_air[b_air["test_id"] == i][y]))
    dataset = b_air[b_air["test_id"] == i][y]
    return dataset

In [19]:
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_cell_hidden_dim, 
                                        forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
    if keep_prob < 1.0:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell

In [20]:
def smape(targets, predictions):
    dividend= np.abs(np.array(targets) - np.array(predictions))
    denominator = np.array(targets) + np.array(predictions)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

In [21]:
def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np-x_np.min()) / (x_np.max()-x_np.min() + 1e-7)

In [28]:
# 하이퍼파라미터
rnn_cell_hidden_dim = 32   # 각 셀의 (hidden)출력 크기
forget_bias = 1.0          # 망각편향(기본값 1.0)
num_stacked_layers = 1     # stacked LSTM layers 개수
keep_prob = 1.0            # dropout할 때 keep할 비율
epoch_num = 1000           # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.01   # 학습률

In [29]:
tf.reset_default_graph()

In [38]:
submission = []
y = 'O3'
for idx,i in enumerate(bj_test_station):
    mean = np.mean(pd.DataFrame(function1(i,y)))
    aq1 = list(pd.DataFrame(function1(i,y)).fillna(round(mean,1))[y])
    #aq = min_max_scaling(aq1)
    
    data=[]
    for i in range(len(aq1)-48):
        aq = aq1[i:i+48]
        data.append(aq)
    
    data_X=[]
    data_Y=[]
    seq_length = 7
    for i in range(len(data)-seq_length):
        data_X.append(data[i:i+seq_length]) 
        data_Y.append(data[i+seq_length]) # i+seq_length번째 날 예측
        
        
    train_size = int(len(data_Y) * 0.8)
    # 데이터를 잘라 학습용 데이터 생성
    trainX = data_X[:train_size]
    trainY = data_Y[:train_size]

    # 데이터를 잘라 테스트용 데이터 생성
    testX = data_X[train_size:]
    testY = data_Y[train_size:]
    
    trainX = np.reshape(trainX, [len(trainX),seq_length,48])
    testX = np.reshape(testX, [len(testX),seq_length,48])
    trainY = np.reshape(trainY,[len(trainY),48])
    testY = np.reshape(testY,[len(testY),48])

    # 모델
    X = tf.placeholder(tf.float32, [None, seq_length, 48])
    Y = tf.placeholder(tf.float32, [None, 48])

    targets = tf.placeholder(tf.float32, [None, 48])
    predictions = tf.placeholder(tf.float32, [None, 48])

    # num_stacked_layers개의 층으로 쌓인 Stacked RNNs 생성
    stackedRNNs = [lstm_cell() for _ in range(num_stacked_layers)]
    multi_cells = tf.contrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()

    # RNN Cell(여기서는 LSTM셀임)들을 연결
    hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
    print("hypothesis: ", hypothesis)

    hypothesis = tf.contrib.layers.fully_connected(hypothesis[:, -1], 48, activation_fn=tf.nn.relu)
    
    # 손실함수로 평균제곱오차를 사용한다
    loss = tf.reduce_sum(tf.square(hypothesis - Y))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    
    #rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets, predictions)))
    temp = tf.divide(tf.abs(targets-predictions), targets+predictions)
    smape = 2*tf.reduce_mean(temp)
    
    train_error_summary = []
    test_error_summary = []  # 테스트용 데이터의 오류를 중간 중간 기록한다
    test_predict = ''        # 테스트용데이터로 예측한 결과
 
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # 학습한다
    print('학습을 시작합니다...')

    for epoch in range(epoch_num):
        _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
        if ((epoch+1) % 100 == 0) or (epoch == epoch_num-1): # 100번째마다 또는 마지막 epoch인 경우

            # 학습용데이터로 rmse오차를 구한다
            train_predict = sess.run(hypothesis, feed_dict={X: trainX})
            train_error = sess.run(smape, feed_dict={targets: trainY, predictions: train_predict})
            train_error_summary.append(train_error)
 
            # 테스트용데이터로 rmse오차를 구한다
            test_predict = sess.run(hypothesis, feed_dict={X: testX})
            test_error = sess.run(smape, feed_dict={targets: testY, predictions: test_predict})
            test_error_summary.append(test_error)
        
            print("epoch: {}, train_error: {}, test_error: {}".format(epoch+1,train_error,test_error))
    
    recent_data = np.reshape(data_X[len(data_X)-seq_length:][0], [-1,seq_length,48])
    test_predict = sess.run(hypothesis, feed_dict={X: recent_data})
    #print("test_predict", test_predict)
     
    submission.append(test_predict[0])
    
    tf.reset_default_graph()

hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 1.101232886314392, test_error: 1.2101248502731323
epoch: 200, train_error: 0.7825236916542053, test_error: 0.9217275381088257
epoch: 300, train_error: 0.6525397896766663, test_error: 0.7970860004425049
epoch: 400, train_error: 0.5983300805091858, test_error: 0.7338069677352905
epoch: 500, train_error: 0.5780307054519653, test_error: 0.7010804414749146
epoch: 600, train_error: 0.570580244064331, test_error: 0.6840179562568665
epoch: 700, train_error: 0.5683408975601196, test_error: 0.6756871342658997
epoch: 800, train_error: 0.5676265954971313, test_error: 0.6714863777160645
epoch: 900, train_error: 0.5674375891685486, test_error: 0.669567346572876
epoch: 1000, train_error: 0.5673786401748657, test_error: 0.6687657237052917
hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 1.0699470043182373, test_error: 1.091696023

학습을 시작합니다...
epoch: 100, train_error: 1.218104600906372, test_error: 1.2420389652252197
epoch: 200, train_error: 0.8749979734420776, test_error: 0.9572153687477112
epoch: 300, train_error: 0.6990376114845276, test_error: 0.8078488111495972
epoch: 400, train_error: 0.6074635982513428, test_error: 0.7310717701911926
epoch: 500, train_error: 0.5603134632110596, test_error: 0.6903473138809204
epoch: 600, train_error: 0.5350787043571472, test_error: 0.6655907034873962
epoch: 700, train_error: 0.5220001339912415, test_error: 0.6543473601341248
epoch: 800, train_error: 0.5160124897956848, test_error: 0.6483049392700195
epoch: 900, train_error: 0.5130209922790527, test_error: 0.6451417803764343
epoch: 1000, train_error: 0.5115892887115479, test_error: 0.643268346786499
hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 1.1854407787322998, test_error: 1.2213962078094482
epoch: 200, train_error: 0.9392066597938538, test_error: 1.018997

epoch: 900, train_error: 0.43076980113983154, test_error: 0.501844048500061
epoch: 1000, train_error: 0.43182265758514404, test_error: 0.4975598454475403
hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 1.1860820055007935, test_error: 1.257865309715271
epoch: 200, train_error: 0.8285170197486877, test_error: 0.8941126465797424
epoch: 300, train_error: 0.65833979845047, test_error: 0.7420116066932678
epoch: 400, train_error: 0.5831578373908997, test_error: 0.663076639175415
epoch: 500, train_error: 0.5513033866882324, test_error: 0.6230135560035706
epoch: 600, train_error: 0.537195086479187, test_error: 0.602410078048706
epoch: 700, train_error: 0.5315011143684387, test_error: 0.5925320386886597
epoch: 800, train_error: 0.5297257900238037, test_error: 0.5877115726470947
epoch: 900, train_error: 0.5292105078697205, test_error: 0.5855441689491272
epoch: 1000, train_error: 0.5290820002555847, test_error: 0.5845286250114441
hypo

epoch: 600, train_error: 0.7265198230743408, test_error: 0.8213872909545898
epoch: 700, train_error: 0.7265394926071167, test_error: 0.8169547319412231
epoch: 800, train_error: 0.7266800999641418, test_error: 0.8150802254676819
epoch: 900, train_error: 0.7267462611198425, test_error: 0.8143636584281921
epoch: 1000, train_error: 0.7267696857452393, test_error: 0.8141176104545593
hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 1.0572776794433594, test_error: 1.1452678442001343
epoch: 200, train_error: 0.7867748737335205, test_error: 0.9022552371025085
epoch: 300, train_error: 0.6806116700172424, test_error: 0.8011801242828369
epoch: 400, train_error: 0.6410099864006042, test_error: 0.7572517991065979
epoch: 500, train_error: 0.6261773705482483, test_error: 0.737678050994873
epoch: 600, train_error: 0.6203538179397583, test_error: 0.7282134890556335
epoch: 700, train_error: 0.6181263327598572, test_error: 0.7237843871116638
e

In [35]:
submission[8]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [36]:
# nan....
#submission[8]
submission[8] = np.array(np.mean(pd.DataFrame(submission).fillna(0)))

In [33]:
PM25 = np.reshape(submission,-1)
PM25

array([86.42341614, 86.7063446 , 87.58744049, ..., 79.71664429,
       78.56103516, 77.88911438])

In [37]:
PM10 = np.reshape(submission,-1)
PM10

array([124.87099457, 124.52959442, 124.61665344, ..., 138.3986969 ,
       138.23973083, 138.08988953])

In [42]:
O3 = np.reshape(submission,-1)
O3

array([73.00096 , 72.924324, 73.4764  , ..., 69.26859 , 54.674362,
       40.458294], dtype=float32)

In [43]:
pm25 = pd.DataFrame(PM25)
pm10 = pd.DataFrame(PM10)
o3 = pd.DataFrame(O3)

In [44]:
pm25.to_csv('pm25_0515.csv')
pm10.to_csv('pm10_0515.csv')
o3.to_csv('o3_0515.csv')

In [45]:
p25 = pd.read_csv('ld_25_0515.csv')
p10 = pd.read_csv('ld_10_0515.csv')

In [46]:
pm25.columns = ['y']
pm10.columns = ['y']
o3.columns = ['y']
p25.columns = ['a', 'y']
p10.columns = ['a', 'y']

In [47]:
len(p25)

624

In [48]:
sample['PM2.5'] = list(pm25['y'])+(list(p25['y']))

In [49]:
sample['PM10'] = list(pm10['y'])+(list(p10['y']))

In [50]:
sample['O3'][:1680] = list(o3['y'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
bj_test_station[8]

'zhiwuyuan_aq'

In [52]:
sample.to_csv('sample_0515.csv',index=False)

In [53]:
import requests
files = {'files': open('sample_0515.csv','rb')}
data = {
    "user_id": "dasolhwang",
    "team_token": "53ed3560830874472182ca33790fe582ebd15bfb2f0506cb03d7757a9acc7480",
    "description": 'dasol_2018-05-15',
    "filename": "sample_0515.csv",
}
url = 'https://biendata.com/competition/kdd_2018_submit/'
response = requests.post(url, files=files, data=data)
print(response.text)

{"success": true}
